# Analysis for small business loans issued for Covid relief

In [1]:
# import packages of interest
import numpy as np
import pandas as pd

In [2]:
# Doing some visualization set up for jupyter notebook 
pd.set_option("display.max_columns", 200)

pd.set_option("display.max_rows", 50000)

In [3]:
# importing and Reading CSV file
df = pd.read_csv('PPP Data 150k plus.csv')
NAICS = pd.read_csv('2017_to_2012_NAICS.csv',sep=';')

In [4]:
# Analysing the first lines of the data
df.head()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
0,a $5-10 million,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723.0,813920.0,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,295.0,04/14/2020,"National Cooperative Bank, National Association",AK - 00
1,a $5-10 million,CRUZ CONSTRUCTION INC,7000 East Palmer Wasilla Hwy,PALMER,AK,99645.0,238190.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,215.0,04/15/2020,First National Bank Alaska,AK - 00
2,a $5-10 million,"I. C. E. SERVICES, INC",2606 C Street,ANCHORAGE,AK,99503.0,722310.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,367.0,04/11/2020,KeyBank National Association,AK - 00
3,a $5-10 million,KATMAI HEALTH SERVICES LLC,"11001 O'MALLEY CENTRE DRIVE, SUITE 204",ANCHORAGE,AK,99515.0,621111.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,0.0,04/29/2020,Truist Bank d/b/a Branch Banking & Trust Co,AK - 00
4,a $5-10 million,MATANUSKA TELEPHONE ASSOCIATION,1740 S. CHUGACH ST,PALMER,AK,99645.0,517311.0,Cooperative,Unanswered,Unanswered,Unanswered,NaN,267.0,06/10/2020,CoBank ACB,AK - 00


In [5]:
# Creating function/tool to analyse the data.
def auxiliar_df(df):
    df_aux = pd.DataFrame({'columns':df.columns,
                           'type':df.dtypes,
                           'missing':df.isnull().sum(),
                           'size': df.shape[0],
                           'unique':df.nunique()})
    df_aux['missing_%']=round(df_aux['missing']/df_aux['size'],2)
    return df_aux

df_aux = auxiliar_df(df)
df_aux

,columns,type,missing,size,unique,missing_%
LoanRange,LoanRange,object,0,661218,5,0.00
BusinessName,BusinessName,object,8,661218,656592,0.00
Address,Address,object,17,661218,628512,0.00
City,City,object,15,661218,15791,0.00
State,State,object,0,661218,57,0.00
Zip,Zip,float64,16,661218,25349,0.00
NAICSCode,NAICSCode,float64,6783,661218,1167,0.01
BusinessType,BusinessType,object,1429,661218,17,0.00
RaceEthnicity,RaceEthnicity,object,0,661218,7,0.00
Gender,Gender,object,0,661218,3,0.00


# EDA

### Creating a column for the loan amount value:

In [6]:
df['LoanType'] = df['LoanRange'].str[0]

In [7]:
mapping = {'a':'5-10',
       'b':'2-5',
       'c':'1-2',
       'd':'0.35-1',
       'e':'0.15-0.35'}

df['LoanAmount'] = df['LoanType'].map(mapping)

### there is 94% of missing value in the column `NonProfit`. Therefore it is a good idea to just drop the column 

In [8]:
df.drop('NonProfit',axis=1,inplace = True)

### there are less than 1% of missing values for the columns `City`, `Address`, `Zip`, and `BusinessType`. Therefore we can just use `dropna` for those columns: 

In [9]:
df=df.dropna(axis=0,how='any',subset = ['City','Address','Zip','BusinessType'])

### I've found out that there some duplicates entries for some business, so I used the `drop_duplicates` for the columns 'BusinessName'

In [15]:
df = df.drop_duplicates(subset=['BusinessName'])

# Let's create a column for business industry

In the dataset `df` we have the column `NAICSCode` which is a code that represents which business industry the company is part of.
Therefore, to find out on which industry the company is part of, I download a dataset from official government resources which contains the industry name and its NAISCode. After that I created a dictionary using these two columns where I like to use as a "Look up table".

With the dictionary created I can use the `pd.map()` function to create a columns with the business industry for each company.

In [11]:
#dic
NAICS = NAICS[['2017 NAICS Code','2017 NAICS Title']]
dic = NAICS.set_index('2017 NAICS Code').T.to_dict('list')
NAICS_dic={}
for x,i in zip(dic.keys(),dic.values()):
    NAICS_dic[x] = i[0]


<ipython-input-11-a63ee44206b5>:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dic = NAICS.set_index('2017 NAICS Code').T.to_dict('list')


In [12]:
df['BusinessIndustry'] = df['NAICSCode'].map(NAICS_dic)

In [13]:
df_aux = auxiliar_df(df)
df_aux

,columns,type,missing,size,unique,missing_%
LoanRange,LoanRange,object,0,655192,5,0.00
BusinessName,BusinessName,object,1,655192,655191,0.00
Address,Address,object,0,655192,623071,0.00
City,City,object,0,655192,15736,0.00
State,State,object,0,655192,57,0.00
Zip,Zip,float64,0,655192,25298,0.00
NAICSCode,NAICSCode,float64,6623,655192,1167,0.01
BusinessType,BusinessType,object,0,655192,17,0.00
RaceEthnicity,RaceEthnicity,object,0,655192,7,0.00
Gender,Gender,object,0,655192,3,0.00


### Exporting the df to csv:

In [16]:
df.to_csv('df_clean.csv',index=False)

# Important:

### I am just cleaning the data using python, I will analyse it using the visualization tool TABLEAU. Follow the link below for the dashboard:

https://public.tableau.com/profile/sylvio.gubitoso#!/vizhome/AnalysisforsmallbusinessloansissuedforCovidrelief/Dashboard1?publish=yes